## __LISA ORBITS --> MEM INPUT FILE --> MEM OUTPUT ANALYSIS__
---
##### __Author:__ Wiler Arturo Sanchez
##### __Affiliation:__ Precision Space Systems Laboratory, University of Florida
##### __Objective__: The LISA Orbits toolbox allows users to generate orbit files of the LISA constellation. NASA's Meteoroid Environment Model allows users to generate the meteoroid environment of a given spacecraft trajectory. This notebook aims to connect both LISA Orbit and NASA MEM via commandline. Doing so will enable us to be conduct a full analysis of the meteoroid environment for LISA.<br>

In [47]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import lisaorbits
import h5py

---

### 1. Initialize LISA Orbit parameters

In [49]:
duration = 31536000                               # a year in seconds
size = 1000                                       # number of elements
t_vec = np.linspace(0, duration, size)            # time vector
#time_origin = datetime.datetime(2034, 1, 1, 0, 0) # approximate start of LISA mission
orbits = lisaorbits.KeplerianOrbits(size = size, dt = int(duration/size))
file = "LISA_KEPLER_ORBIT_1YR_SIZE1e3.h5"
#orbits.write("LISA_KEPLER_ORBIT_1YR_SIZE1e3.h5")

---

### 2. Ensure orbit file is generated correctly

In [ ]:
%matplotlib widget
#orbits.plot_spacecraft(1)
#orbits.plot_spacecraft(2)
#orbits.plot_spacecraft(3)
#orbits.plot_links()

---

### 3. Initialize the position and velocity state vectors from orbit file

In [50]:
sc_vec = ['1', '2', '3']     # spacecraft number
dimensions = ['x', 'y', 'z'] # spacecraft axes
dyn = ['x', 'v', 'a']        # position, velocity, acceleration
size = 1000                  # initialize number of elements 

dynamics = {}                # construct dictionary to store all values

# simple for loop to place sc dynamics vectors into dictionary
for i, dyn in enumerate(dyn):
    for j, scvec in enumerate(sc_vec):
        for k, dim in enumerate(dimensions):
            with h5py.File(f'./{file}', 'r') as f:
                # barycentric coordinate time
                # convert from distance units from km to m
                # [number of elements, sc#, spacecraft axis]
                dynamics[f'{scvec}_{dim}_{dyn}'] = f[f'/tcb/{dyn}'][:, j, k]/1000 

#### Verify shape

In [ ]:
dynamics['1_x_x'].shape

---

### 4. Format output file to fit the requirements of MEM input file

In [ ]:
# Define the Julian date range with 1000 steps
start_jd = 2463964    # January 1st, 2022 (must change to 2034)
end_jd   = 2464329    # January 1st, 2023
jd_range = np.linspace(start_jd, end_jd, 1000)

output_dir = '/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/'

# Write data to text files for each spacecraft
for i, scvec in enumerate(sc_vec):
    # Define file name
    file_name = os.path.join(output_dir, f"LISATRAJECTORY_sc_{scvec}.txt")
    with open(file_name, 'w') as f:
            f.write("\n\n\n\n\n")
            f.write(f"Julian Date (days)   sc_{scvec}_X (km)   sc_{scvec}_Y (km)   sc_{scvec}_Z (km)   sc_{scvec}_V_X (km/s)   sc_{scvec}_V_Y (km/s)   sc_{scvec}_V_Z (km/s)\n")
            for k, jd in enumerate(jd_range):
                    f.write(f"{jd:.5f}   {scx[i][0][k]:.5f}   {scx[i][1][k]:.5f}   {scx[i][2][k]:.5f}   {scv[i][0][k]:.5f}   {scv[i][1][k]:.5f}   {scv[i][2][k]:.5f}\n")
                    for j, dim in enumerate(dimensions):
                        df = pd.DataFrame({'Julian Date (days)': jd_range,
                                            f'sc_{i+1}_x (km)': scx[i][j][k],
                                            f'sc_{i+1}_y (km)': scx[i][j][k],
                                            f'sc_{i+1}_z (km)': scx[i][j][k],
                                            f'sc_{i+1}_vx (km/s)': scv[i][j][k],
                                            f'sc_{i+1}_vy (km/s)': scv[i][j][k],
                                            f'sc_{i+1}_vz (km/s)': scv[i][j][k]})

---

### 5. Write *options.txt* file with desired input parameters

In [ ]:
import os
!pwd

In [ ]:
os.chdir('/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/')

In [61]:
# create options file with the following text
input_files = ["LISATRAJECTORY_sc_1", "LISATRAJECTORY_sc_2", "LISATRAJECTORY_sc_3"]
input_origin = 'su'
input_axes   = 'eq'
state_vector = 'a'
number_of_sv = '0'
mass_min     = '-6'
option_fidelity = 'y'
fidelity = 'high fidelity'
output_directory = str(f'C:\\LISA_simulation\\workspace\\MEM3_Windows_301\\MEM3_Windows\\')
output_file = 'LISA_sc1_04_18_2023'
output_origin = 'su'
output_axes = 'bo'
desired_output_angres = '5'
desired_output_velres = '1'
option_intfiles = 'n'
output_intfiles = 'intermediate files'
option_igloo = 'y'
output_igloo = 'igloo files'
option_std = 'y'
output_std = 'standard deviations'

# Write to a text file
for i, input_file in enumerate(input_files):
    with open('options.txt', 'w') as f:
        f.write('#' * 80 + '\n')
        f.write('# Input file for the command line version of MEM 3, must be named "options.txt"\n')
        f.write('# The following files must be located in the same directory :\n')
        f.write('#  - MEM3Windows.exe (Windows) or mem3cl (Mac or Linux)\n')
        f.write('#  - bin*.dat (3 files)\n')
        f.write('#  - de430\n')
        f.write('#  - options.txt, rho1.txt, and rho2.txt\n')
        f.write('# This file created by MEM3GUI version 3.0\n')
        f.write('#' * 80 + '\n')
        f.write('# This input file must contain the following lines\n')
        f.write('# Line 1 - name of input file\n')
        f.write('# Line 2 - input origin : (su)n, (ea)rth, (mo)on, (me)rcury, (ve)nus, (ma)rs\n')
        f.write('# Line 3 - input axes : (eq)uatorial or (ec)liptic\n')
        f.write('# Line 4 - use(a)ll state vectors, a(r)andom selection, or the(f)irst n\n')
        f.write('# Line 5 - integer number of state vectors to be used(ignored if using all)\n')
        f.write('# Line 6 - log10 of the desired minimum particle mass, within[-6, 1]\n')
        f.write('# Line 7 - high fidelity run(y for yes)\n')
        f.write('# Line 8 - directory within which results will be placed\n')
        f.write('# Line 9 - run name(will be used to name results folder)\n')
        f.write('# Line 10 - output origin : (su)n, (ea)rth, (mo)on, (me)rcury, (ve)nus, (ma)rs\n')
        f.write('# Line 11 - output axes : (eq)uatorial, (ec)liptic, or (bo)dy - fixed\n')
        f.write('# Line 12 - desired output resolution in degrees(1, 2, 3, 4, or 5)\n')
        f.write('# Line 13 - desired output resolution in km / s(1 or 2)\n')
        f.write('# Line 14 - output intermediate files(y for yes)\n')
        f.write('# Line 15 - output igloo files(y for yes)\n')
        f.write('# Line 16 - output standard deviation files(y for yes)\n')
        f.write('#\n')
        f.write('#' * 80 + '\n')
        input_directory = str(f'C:\\LISA_simulation\\workspace\\MEM3_Windows_301\\MEM3_Windows\\{input_files[i]}.txt')
        f.write(f'{input_directory}\n')
        f.write(f'{input_origin}\n')
        f.write(f'{input_axes}\n')
        f.write(f'{state_vector}\n')
        f.write(f'{number_of_sv}\n')
        f.write(f'{mass_min}\n')
        f.write(f'{option_fidelity} {fidelity}\n')
        f.write(f'{output_directory}\n')
        f.write(f'{output_file}\n')
        f.write(f'{output_origin}\n')
        f.write(f'{output_axes}\n')
        f.write(f'{desired_output_angres}\n')
        f.write(f'{desired_output_velres}\n')
        f.write(f'{option_intfiles} {output_intfiles}\n')
        f.write(f'{option_igloo} {output_igloo}\n')
        f.write(f'{option_std} {output_std}\n')

#### Run MEM

In [ ]:
# linux version of MEM required
!/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/MEM3Windows.exe

---

### 6. Load results and plot the Meteoroid Flux distribution on a Mollweide projection

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf

files               = ['LISA_sc1_04_18_2023','LISA_sc2_04_30_2023','LISA_sc3_04_30_2023']
densities           = ['HiDensity','LoDensity']
spacecraft_names    = ['Spacecraft 1','Spacecraft 2','Spacecraft 3']
density_mean_values = ['3.579','2.933']
density_std_values  = ['0.093','0.127']

#pdf = matplotlib.backends.backend_pdf.PdfPages('plots_05_14_2023.pdf')

for i, name in enumerate(files):
    for j, density in enumerate(densities):
        fig, ax = plt.subplots(figsize=(12, 8), dpi = 120, facecolor='w', subplot_kw={'projection': 'mollweide'})

        # Load in density and igloo data
        density_data    = pd.read_table(f"/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/{name}/HiDensity/igloo_avg.txt",sep="\s+", header = 7)
        igloo_data      = pd.read_table(f"/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/{name}/{density}/igloo_avg.txt",sep="\s+", header = 7)
        phi             = pd.Series(np.deg2rad(igloo_data['THETA2'][:]))
        theta           = pd.Series(np.deg2rad(igloo_data['PHIavg'][:] - 180)) # offset by 180 deg to correct the plot
        velocity_bins   = pd.concat([pd.Series(['THETAavg']), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True)
        flux_vs_vel_matrix = np.zeros([80, 1652])
        
        for ii in range(len(velocity_bins)): # velocity distribution
            for jj in range(1652): # 
                flux_vs_vel_matrix[ii][jj] = igloo_data[str(velocity_bins[ii])][jj]
                
        # Plot the data as a scatter plot
        index = 23 # c=fluxM[i, j, k]     = fluxV[i, j] * dgrun[k]
        sc = ax.scatter(theta, phi, c=flux_vs_vel_matrix[index, :], cmap='viridis', alpha=0.7, s=100, lw=0)
        
        # Set the axis labels and title
        ax.set_xlabel(r'Azimuth, $\theta$', fontsize=14, fontweight='bold')
        ax.set_ylabel(r'Elevation, $\phi$', fontsize=14, fontweight='bold')
        ax.set_title(f'{density} Population, $\mu = {density_mean_values[j]}$ and $\sigma = {density_std_values[j]}$ [$kg/m^3$]\nMeteoroid Flux Distribution of LISA {spacecraft_names[i]} for ~{velocity_bins[index]} km/s', fontsize=16, fontweight='bold')
        ax.set_xticklabels(['-150°', 'starboard\n-120°', '-90°', '-60°', '-30°', 'zenith\n\n\n\n\n\n\n\n\n\n\n\nram\n0°', '30°', '60°', '90°', 'port\n120°', '150°'],
                            fontsize=12, fontweight='bold')
        ax.set_yticklabels(['-75°', '-60°', '-45°', '-30°', '-15°', '0°', '15°', '30°', '45°', '60°', '75°'],
                            fontsize=12, fontweight='bold')
        ax.text(-0.35,-1.35, "nadir",fontsize=12, fontweight='bold')

        # Add a colorbar
        cbar = plt.colorbar(sc)
        cbar.set_label('Flux [# of particles/m^2*yr]')
        fig.tight_layout()
        plt.grid(True)
        plt.savefig(f'{name}_{density}',dpi=200)
        plt.show()
        

---

### 7. Plots of the Meteoroid Flux distribution at all velocity bins

In [ ]:
%matplotlib inline
import matplotlib.colors as colors

for i, name in enumerate(files):
    for j, density in enumerate(densities):
        # Load in density and igloo data
        density_data       = pd.read_table(f"/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/{name}/HiDensity/igloo_avg.txt",sep="\s+", header = 7)
        igloo_data         = pd.read_table(f"/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/{name}/{density}/igloo_avg.txt",sep="\s+", header = 7)
        phi                = pd.Series(np.deg2rad(igloo_data['THETA2'][:]))
        theta              = pd.Series(np.deg2rad(igloo_data['PHIavg'][:] - 180)) #offset by 180 deg to correct the plot
        velocity_bins      = pd.concat([pd.Series(['THETAavg']), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True)
        flux_vs_vel_matrix = np.zeros([80, 1652])
        
        for ii in range(len(velocity_bins)): # velocity distribution
            for jj in range(1652): # 
                flux_vs_vel_matrix[ii][jj] = igloo_data[str(velocity_bins[ii])][jj]
        
        velocity_bins      = pd.concat([pd.Series(['THETAavg']), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True)
        vbins              = np.array(pd.concat([pd.Series([0]), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True))

        size = 80

        fig, axs = plt.subplots(nrows=16, ncols=5, figsize=(12, 20), dpi=120, subplot_kw={'projection': 'mollweide'}, facecolor='w')

        for uu, ax in enumerate(axs.flat):
            if uu >= igloo_wide.to_numpy().shape[1]:
                break
            ax.scatter(theta, phi, c=flux_vs_vel_matrix[uu, :], cmap='viridis', alpha=1, s=10, lw=0)
            ax.set_title(f'{vbins[uu]} km/s')
            ax.set_xticks([])
            ax.set_yticks([])

        fig.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95, wspace=0.3, hspace=0.3)
        #fig.suptitle(f'{density} Population\nFlux Distribution of LISA science {spacecraft_names[i]} at each meteoroid velocity bin', fontsize=20)
        plt.tight_layout()
        #plt.savefig(f'fullfluxdist_{name}_{density}',dpi=200)
        plt.show()
    

---

### 8. Histogram plots of flux vs velocity

In [ ]:
%matplotlib widget
import matplotlib.colors as colors

for i, name in enumerate(files):
    fig, ax = plt.subplots(figsize=(12, 8), dpi = 120, facecolor='w')
    for j, density in enumerate(densities):
        # Load in density and igloo data
        density_data    = pd.read_table(f"/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/{name}/{density}/igloo_avg.txt",sep="\s+", header = 7)
        igloo_data      = pd.read_table(f"/home/lisauser/workspace/MEM3_Windows_301/MEM3_Windows/{name}/{density}/igloo_avg.txt",sep="\s+", header = 7)
        phi             = pd.Series(np.deg2rad(igloo_data['THETA2'][:]))
        theta           = pd.Series(np.deg2rad(igloo_data['PHIavg'][:] - 180)) #offset by 180 deg to correct the plot
        velocity_bins   = pd.concat([pd.Series(['THETAavg']), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True)
        
        flux_vs_vel_matrix = np.zeros([79, 1652])
        for ii in range(len(velocity_bins)): # velocity distribution
            for jj in range(1652): # 
                flux_vs_vel_matrix[ii][jj] = igloo_data[str(velocity_bins[ii])][jj]
        
        # calculate average flux per velocity
        avg_flux_vs_vel = np.sum(flux_vs_vel_matrix, axis=1) / flux_vs_vel_matrix.shape[0]
           
        # Plot 2D histogram in separate figures based on spacecraft
        ax.plot(vbins, avg_flux_vs_vel, label=f'{density}')
        
    # Configure and show the figure for the current density
    ax.set_xlabel('Meteoroid Speed [km/s]')
    ax.set_ylabel('Average Flux [# of particles/m^2*yr]')
    ax.set_title(f'{spacecraft_names[i]}\nDistribution of Meteoroid Flux vs. Speed')
    ax.legend()
    plt.savefig(f'histograms_{name}',dpi=200)
    plt.show()
    #pdf.savefig(fig)
    #plt.close(fig)  # Close the figure to free up memory

---

### 9. Plots of the momentum distribution

#### Import libraries and dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import loguniform
from sklearn.preprocessing import normalize

#### Load constants and file names

In [ ]:
files               = ['LISA_sc1_04_18_2023','LISA_sc2_04_30_2023','LISA_sc3_04_30_2023']
densities           = ['HiDensity','LoDensity']
spacecraft_names    = ['Spacecraft 1','Spacecraft 2','Spacecraft 3']
density_mean_values = ['3.579','2.933']
density_std_values  = ['0.093','0.127']

#### Define all functions for this section

##### The *grunmodel* function takes in an array of masses and outputs a flux array with each element of the mass array corresponding to a flux value.<br> This function was extracted from the MEM documentation.

In [ ]:
def GrunModel(m):
    C     = np.array([2.2e3, 15, 1.3e-9, 1e11, 1e27, 1.3e-16, 1e6])
    g     = np.array([0.306, -4.38, 2, 4, -0.36, 2, -0.85])

    G = ((C[0]*(m**g[0])) + C[1])**g[1] + \
           C[2]*(m + C[3]*(m**g[2]) + C[4]*(m**g[3]))**g[4] + \
           C[5]*(m + C[6]*(m**g[5]))**g[6]
    
    return G

##### This *InverseGrunModel* function is the inverse of the above function. Computed using Wolfram Mathematica.

In [ ]:
def InverseGrunModel(G):
    C     = np.array([2.2 * 10**3, 15, 1.3 * 10**-9, 10**11, 10**27, 1.3 * 10**-16, 10**6])
    g     = np.array([0.306, -4.38, 2, 4, -0.36, 2, -0.85])
    
    t1 = C[0] * g[0] * g[1] * G**(-1 + g[0]) * (C[1] + C[0] * G**g[0])**(-1 + g[1])
    t2 = C[2] * g[4] * (1 + C[3] * g[2] * G**(-1 + g[2]) + C[4] * g[3] * G**(-1 + g[3])) * (G + C[3] * G**g[2] + C[4] * G**g[3])**(-1 + g[4])
    t3 = C[5] * g[6] * (1 + C[6] * g[5] * G**(-1 + g[5])) * (G + C[6] * G**g[5])**(-1 + g[6])
    
    m = t1 + t2 + t3
    
    return m

##### This *integrate_array* function takes a given array and computes the integrated values for add up to a certain target value.

In [ ]:
def integrate_array(array, target_value):
    cumulative_sum    = np.cumsum(array)
    integrated_values = cumulative_sum[cumulative_sum <= target_value]
    return integrated_values

##### This *sort_sections* function takes in a section of an array and sorts only that section, segment size is specified as one of the arguments

In [ ]:
def sort_sections(array, segment_size):
    sorted_array = array.copy()                          # Create a copy of the original array to preserve the original data
    
    for i in range(0, len(array), segment_size):
        section = sorted_array[i:i+segment_size]         # Get the current section of the array
        sorted_section = sorted(section)                 # Sort the section
        sorted_array[i:i+segment_size] = sorted_section  # Replace the section in the sorted array
        
    return sorted_array

##### Since the grun model takes into account limiting mass, we have to correct the data by subtracting the forward element with the present element. This *process_sections* does this as a function of the array and segment size. Note, each section is prepended with a zero to take into account the amount of elements that are in the array after it has been processed.

In [ ]:
def process_sections(array, segment_size):
    processed_array = []

    for i in range(0, len(array), segment_size):
        section = array[i:i+segment_size]                                       # Get the current section of the array
        
        # Perform the operation on the section
        corrected_section = [section[ii] - section[ii+1] for ii in range(segment_size-1)]
        
        processed_section = [0] + corrected_section                             # Prepend a zero to the section
        processed_section = processed_section/np.sum(processed_section)         # Divide by the sum
        processed_array.extend(processed_section)                               # Append the processed section to the result

    return processed_array

In [ ]:
names = "ID   I   J   PHI1   PHI2 THETA1 THETA2 PHIavg THETAavg        0.5          1.5          2.5          3.5          4.5          5.5          6.5          7.5          8.5          9.5         10.5         11.5         12.5         13.5         14.5         15.5         16.5         17.5         18.5         19.5         20.5         21.5         22.5         23.5         24.5         25.5         26.5         27.5         28.5         29.5         30.5         31.5         32.5         33.5         34.5         35.5         36.5         37.5         38.5         39.5         40.5         41.5         42.5         43.5         44.5         45.5         46.5         47.5         48.5         49.5         50.5         51.5         52.5         53.5         54.5         55.5         56.5         57.5         58.5         59.5         60.5         61.5         62.5         63.5         64.5         65.5         66.5         67.5         68.5         69.5         70.5         71.5         72.5         73.5         74.5         75.5         76.5         77.5         78.5         79.5".split()
igloo_data      = pd.read_table(f"LISA_sc1_04_18_2023/HiDensity/igloo_avg.txt",sep="\s+", comment="#", names=names)
igloo_data

In [ ]:
mom_flux["mom_bin"].to_numpy()
np.arange(-0.795, 795.0, 39.75)

In [ ]:
L = [str(x) for x in np.arange(0.5, 79.5, 1).tolist()]
L[0]

In [ ]:
fluxV = igloo_data[L].to_numpy()
fluxV.shape

In [ ]:
from scipy.stats import loguniform
dgrun = np.zeros(num_mass-1)


# Total number of elements in mass array
num_mass = 20

# Grun model mass array sorted in increasing order 
m = np.logspace(-6, 1, num_mass)  # grams

# Insert mass array into grunmodel function
G = GrunModel(m)

# Correct the Grun model matrix by subtracting the mass bins in increasing order
for ll in range(num_mass-1):
    dgrun[ll] = G[ll] - G[ll+1]
    
dgrun /= np.sum(dgrun)

fluxV = igloo_data[L].to_numpy()

for i in range(1652):
    for j in range(79):
        for k in range(num_mass-1):
            fluxM[i, j, k]     = fluxV[i, j] * dgrun[k]

igloo_long = pd.melt(igloo_data, id_vars=["ID","I","J","PHI1","PHI2","THETA1","THETA2","PHIavg","THETAavg"], var_name="velocity", value_name="flux")
igloo_long["velocity"] = igloo_long["velocity"].astype("float64")
igloo_long["flux"].to_numpy()

mass_df = pd.DataFrame({"mass": m, "flux": G})

mom_flux = pd.merge(igloo_long, mass_df, how="cross", suffixes=("_vel","_mass"))
mom_flux["momentum"] = mom_flux["velocity"] * mom_flux["mass"]
mom_flux["mom_bin"] = pd.cut(mom_flux["momentum"], num_mass)

momentumbins = np.arange(-0.795, 795.0, 39.75)

for l in range(numbins - 1):
    for i in range(1652):
        for j in range(79):
            for k in range(size_ofmassarray):
                    if (momentumbins[l] <= momentum_mat[j, k] < momentumbins[l + 1]):
                        # Compute corrected flux value by multiplying fractional Grun model flux with
                        # experimental flux from MEM output file
                        tot[a, b, l]     += fluxM[i, j, k]     
                                
mom_flux["mom_bin"].to_numpy()
mom_flux
mom_sum = mom_flux.groupby(["PHIavg", "THETAavg", "mom_bin"])["flux_vel"].sum().reset_index()
mom_sum

In [ ]:
[m.mid for m in mom_flux["mom_bin"]]

##### Get the header of the file, then read data ignoring comment lines

In [ ]:
!head -n 8 LISA_sc1_04_18_2023/HiDensity/igloo_avg.txt | tail -n 1

In [ ]:
# Create a string that contains all the headers that define the
# variables in the flux_vs_vel_matrix
names_igloo         = "ID   I   J   PHI1   PHI2 THETA1 THETA2 PHIavg THETAavg        0.5          1.5          2.5          3.5          4.5          5.5          6.5          7.5          8.5          9.5         10.5         11.5         12.5         13.5         14.5         15.5         16.5         17.5         18.5         19.5         20.5         21.5         22.5         23.5         24.5         25.5         26.5         27.5         28.5         29.5         30.5         31.5         32.5         33.5         34.5         35.5         36.5         37.5         38.5         39.5         40.5         41.5         42.5         43.5         44.5         45.5         46.5         47.5         48.5         49.5         50.5         51.5         52.5         53.5         54.5         55.5         56.5         57.5         58.5         59.5         60.5         61.5         62.5         63.5         64.5         65.5         66.5         67.5         68.5         69.5         70.5         71.5         72.5         73.5         74.5         75.5         76.5         77.5         78.5         79.5".split()
names_density       = "rho_min rho_max fraction"
files               = ['LISA_sc1_04_18_2023','LISA_sc2_04_30_2023','LISA_sc3_04_30_2023']
densities           = ['HiDensity','LoDensity']
spacecraft_names    = ['Spacecraft 1','Spacecraft 2','Spacecraft 3']
density_mean_values = ['3.579','2.933']
density_std_values  = ['0.093','0.127']

igloo_data     = {}
igloo_data_std = {}
density_data   = {}

# Use the string that was created previously to load in the data
# as a pandas dataframe and split the data based on header name
for i, file in enumerate(files):
    for j, density in enumerate(densities):
        df = pd.read_table(f"{file}/{density}/igloo_avg.txt", sep="\s+", comment="#", names=names_igloo)
        igloo_data.setdefault(i, {})[j] = df
        
        df_std = pd.read_table(f"{file}/{density}/igloo_std.txt", sep="\s+", comment="#", names=names_igloo)
        igloo_data_std.setdefault(i, {})[j] = df_std
        
        df_density = pd.read_table(f"{file}/{density.lower()}.txt", sep="\s+", usecols=range(0,4))
        df_density = df_density.shift(axis=1)
        df_density = df_density.iloc[1:]
        df_density = df_density.drop(df_density.columns[0], axis=1)
        df_density_avg = (np.array(df_density['rho_min']).astype(float) + np.array(df_density['rho_max']).astype(float))*.5
        df_density['rho_avg'] = df_density_avg            
        density_data.setdefault(i, {})[j] = df_density

In [ ]:
LISA_sc1_HiDensity = igloo_data[0][0]
LISA_sc1_LoDensity = igloo_data[0][1]
LISA_sc2_HiDensity = igloo_data[1][0]
LISA_sc2_LoDensity = igloo_data[1][1]
LISA_sc3_HiDensity = igloo_data[2][0]
LISA_sc3_LoDensity = igloo_data[2][1]

LISA_sc1_HiDensity_std = igloo_data_std[0][0]
LISA_sc1_LoDensity_std = igloo_data_std[0][1]
LISA_sc2_HiDensity_std = igloo_data_std[1][0]
LISA_sc2_LoDensity_std = igloo_data_std[1][1]
LISA_sc3_HiDensity_std = igloo_data_std[2][0]
LISA_sc3_LoDensity_std = igloo_data_std[2][1]

LISA_sc1_HiDensity_den = density_data[0][0]
LISA_sc1_LoDensity_den = density_data[0][1]
LISA_sc2_HiDensity_den = density_data[1][0]
LISA_sc2_LoDensity_den = density_data[1][1]
LISA_sc3_HiDensity_den = density_data[2][0]
LISA_sc3_LoDensity_den = density_data[2][1]

In [ ]:
df

In [ ]:
density_data[0][0]['fraction'].astype("float").values
density_data[0][0]['rho_avg'].astype("float").values

In [ ]:
igloo_data[2][1]['39.5'].max()

In [ ]:
momentumbins = np.concatenate((np.logspace(-6,-5, numbins)[:-1], np.logspace(-5, 3, numbins)))
momentumbins

In [ ]:
fluxM     = np.zeros([1652, 80, size_ofmassarray-1])
fluxM[:, :, :]     = [[[fluxV[i, j] * dgrun[k] for k in range(size_ofmassarray-1)] for j in range(80)] for i in range(1652)]

In [ ]:
vbins = np.linspace(0.5, 78.5, 79)
vbins.shape

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 8), dpi = 120, facecolor='w')

colors             = ['red', 'blue', 'green', 'orange', 'purple', 'brown']  
symbols            = ['o', 's', 'D', '^', 'v', 'P']
velocity_bins      = pd.concat([pd.Series(['THETAavg']), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True)
vbins              = np.linspace(0.5, 78.5, 79)
densities          = ['HiDensity', 'LoDensity']
spacecraft         = ['SC1', 'SC2', 'SC3']

# Total number of elements in mass array
size_ofmassarray = 10

# Grun model mass array sorted in increasing order 
bmass = np.logspace(-6, 1, size_ofmassarray)  # grams
     
# Insert mass array into grunmodel function
G = GrunModel(bmass) * (365.25*24*3600) # need to put the grun model into yrs to match output of the MEM

# Correct the Grun model matrix by subtracting the mass bins in increasing order
dgrun = np.zeros(size_ofmassarray-1)
for ll in range(size_ofmassarray-1):
    dgrun[ll] = G[ll] - G[ll+1]   # check
    
dgrun /= np.sum(dgrun)
    
# Initialize total number of desired bins 
numbins = 10

# Logspace momentum bins 
momentumbins = np.logspace(-6, 3, numbins)

# Matrix multiply vertical average velocity bin array with horizontal Grun model mass array: vel (rows) vs. mass (columns)
momentum_mat = vbins.reshape(-1,1)@bmass.reshape(1,-1)

# Initialiaze flux partitioned by average velocity values
fluxV     = np.zeros([1652, 79])
#fluxV_std = np.zeros([1652, 80])

# Initialize flux dependent on velocity matrix with extra dimension the size of the mass array 
fluxM     = np.zeros([1652, 79, size_ofmassarray])
#fluxM_std = np.zeros([1652, 80, size_ofmassarray])

# Initialize the fractional flux from the MEM output files
#fluxRho   = np.zeros([3, 2, len(density_data[0][0]['fraction'].astype("float").values)])

# intialize momentum dependent flux values matrix
tot     = np.zeros([3, 2, len(momentumbins)])
#tot_std = np.zeros([3, 2, len(momentumbins)])

f = 0

# This for loop iterates through LISA spacecraft files
for a, sc in enumerate(spacecraft):
    
    # This for loop iterates through the density populations of the meteoroids
    for b, density in enumerate(densities):

        # Use a double for loop statement to insert values from MEM output files
        # phi, theta position on spherical projection (sky position)
        for jj in range(79): # velocity
            fluxV[:, jj]     = [igloo_data[a][b][str(velocity_bins[jj])][ii] for ii in range(1652)]
            #fluxV_std[:, jj] = [igloo_data_std[a][b][str(velocity_bins[jj])][ii] for ii in range(1652)]
                
        # This triple for loop is to iterate mass dependent flux values into 3-dimensional matrix 
        for j in range(79):
            for k in range(size_ofmassarray-1):
                fluxM[:, j, k]     = [fluxV[i, j] * dgrun[k] for i in range(1652)]
                #fluxM_std[:, j, k] = [fluxV_std[i, j] * dgrun[k] for i in range(1652)]
        
        # This quad for loop attempts to combine the Grun mass model flux with experimental flux from MEM output 
        # files to iterate through a momentum dependent flux matrix. Insert flux values into respective momentum
        # bins.
        for l in range(numbins - 1):
            for i in range(1652):
                for j in range(79):
                    for k in range(size_ofmassarray):
                        if l == 0:
                             if (momentum_mat[j, k] <= momentumbins[l]):
                                tot[a, b, l]     += fluxM[i, j, k]     
                                #tot_std[a, b, l] += fluxM_std[i, j, k]
                        elif l == (numbins - 1):
                             if (momentumbins[l] <= momentum_mat[j, k]):
                                tot[a, b, l]     += fluxM[i, j, k]     
                                #tot_std[a, b, l] += fluxM_std[i, j, k]
                        else: 
                            if (momentumbins[l] <= momentum_mat[j, k] < momentumbins[l + 1]):
                            # Compute corrected flux value by multiplying fractional Grun model flux with
                            # experimental flux from MEM output file
                                tot[a, b, l]     += fluxM[i, j, k]     
                                #tot_std[a, b, l] += fluxM_std[i, j, k] 
                                
        #print(tot[a,b,:])
        # Plot the flux values against the momentum bins. 
        ax.loglog(momentumbins * 1e6, tot[a, b, :], color=colors[f], marker=symbols[f],\
                  markersize = 3, label=f'{sc}_{density}', linewidth = 1)
        f += 1
        #plt.errorbar(momentumbins, tot[a, b, :], yerr=tot_std[a, b, :] /(365.25*24*3600) ,color=colors[f], marker=symbols[f])  
        #ax.set_title(f'Density: {density}, Spacecraft: {sc}')

        
        #ax.set_xlim([10**-5, 10**4])
        #ax.set_ylim([10**-5, 10**8])
        
plt.xlim([1e-1, 1e8])
plt.ylim([1e-8, 1e6])
plt.ylabel(r'Flux $[numparticles*yr^{-1}m^{-2}]$')
plt.xlabel(r'Momentum, $[\mu N s]$') # newton seconds  
plt.legend()
plt.grid(True)
#plt.savefig('momentum distribution.png',dpi=200)
plt.show()

In [ ]:
tot

In [ ]:
numbins = 50 
vbins = 80
momentumbins = np.concatenate((np.logspace(-6,-5, numbins)[:-1], np.logspace(-5, 3, numbins)))
print(tot[0, 0, :])
print(tot[0, 1, :])
print(tot[1, 0, :])
print(tot[1, 1, :])
print(tot[2, 0, :])
print(tot[2, 1, :])

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 10), facecolor='w', dpi=120)
colors             = ['red', 'blue', 'green', 'orange', 'purple', 'brown']  
symbols            = ['o', 's', 'D', '^', 'v', 'P']                    
densities          = ['HiDensity', 'LoDensity']
spacecraft         = ['SC1', 'SC2', 'SC3']
f = 0
# This for loop iterates through LISA spacecraft files
for a, sc in enumerate(spacecraft):
    # This for loop iterates through the density populations of the meteoroids
    for b, density in enumerate(densities):
        ax.scatter(momentumbins * 1e6, tot[a, b, :], color=colors[f],\
                     marker=symbols[f], label=f'{sc}_{density}')
        #ax[a].errorbar(momentumbins, tot[a, b, :] / (365.25*24*3600), yerr = tot_std[a, b, :] /(365.25*24*3600), color=colors[b])
        ax.set_ylabel(fr'Flux, $[num*yr^{-1}m^{-2}]$')
        ax.set_xlabel(r'Momentum, $[\mu N s]$') # newton seconds 
        ax.legend()
        #ax.lines[f].set_label(f'{sc}_{density}')
        ax.set_yscale('log')
        ax.set_xscale('log')
        ax.set_xlim([1e-1, 1e2])
        ax.set_ylim([1e-8, 1e6])
        ax.grid(True)
        f += 1

plt.suptitle('Momentum dependent flux distribution')
plt.tight_layout()
#plt.savefig('Momentum_dependent_flux_distribution.png', dpi=200)
plt.show()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 8), dpi=120)

plt.loglog(momentumbins*1000000, np.abs(tot), 'bo')
ax.set_ylabel(r'Flux $[yr^{-1}m^{-2}]$')
ax.set_xlabel(r'Momentum, P $[\mu Ns]$')
ax.set_xlim([10**2, 10**5])
ax.set_ylim([10**-5, 10**2])
plt.grid()
plt.show()

In [ ]:
momentum_mat[:, 15]

In [ ]:
vbins.reshape(-1,1)

In [ ]:
size = 20
m    = np.logspace(-6, -3, size)

##### Reshape table to make velocity a column using melt. Velocity comes as an object, so cast it to a float instead.

In [ ]:
import pandas as pd
import numpy as np
# Create a string that contains all the headers that define the
# variables in the flux_vs_vel_matrix
names_igloo         = "ID   I   J   PHI1   PHI2 THETA1 THETA2 PHIavg THETAavg        0.5          1.5          2.5          3.5          4.5          5.5          6.5          7.5          8.5          9.5         10.5         11.5         12.5         13.5         14.5         15.5         16.5         17.5         18.5         19.5         20.5         21.5         22.5         23.5         24.5         25.5         26.5         27.5         28.5         29.5         30.5         31.5         32.5         33.5         34.5         35.5         36.5         37.5         38.5         39.5         40.5         41.5         42.5         43.5         44.5         45.5         46.5         47.5         48.5         49.5         50.5         51.5         52.5         53.5         54.5         55.5         56.5         57.5         58.5         59.5         60.5         61.5         62.5         63.5         64.5         65.5         66.5         67.5         68.5         69.5         70.5         71.5         72.5         73.5         74.5         75.5         76.5         77.5         78.5         79.5".split()
names_density       = "rho_min rho_max fraction"
files               = ['LISA_sc1_04_18_2023','LISA_sc2_04_30_2023','LISA_sc3_04_30_2023']
densities           = ['HiDensity','LoDensity']
spacecraft_names    = ['Spacecraft 1','Spacecraft 2','Spacecraft 3']
density_mean_values = ['3.579','2.933']
density_std_values  = ['0.093','0.127']
vbins              = np.array(pd.concat([pd.Series([0]), pd.Series(np.linspace(0.5, 78.5, 79))], ignore_index=True))

igloo_data     = {}
igloo_data_std = {}
density_data   = {}

# Use the string that was created previously to load in the data
# as a pandas dataframe and split the data based on header name
for i, file in enumerate(files):
    for j, density in enumerate(densities):
        df = pd.read_table(f"{file}/{density}/igloo_avg.txt", sep="\s+", comment="#", names=names_igloo)
        igloo_data.setdefault(i, {})[j] = df
        
        df_std = pd.read_table(f"{file}/{density}/igloo_std.txt", sep="\s+", comment="#", names=names_igloo)
        igloo_data_std.setdefault(i, {})[j] = df_std
        
        df_density = pd.read_table(f"{file}/{density.lower()}.txt", sep="\s+", usecols=range(0,4))
        df_density = df_density.shift(axis=1)
        df_density = df_density.iloc[1:]
        df_density = df_density.drop(df_density.columns[0], axis=1)
        df_density_avg = (np.array(df_density['rho_min']).astype(float) + np.array(df_density['rho_max']).astype(float))*.5
        df_density['rho_avg'] = df_density_avg            
        density_data.setdefault(i, {})[j] = df_density

In [ ]:
LISA_sc1_HiDensity = igloo_data[0][0]
LISA_sc1_LoDensity = igloo_data[0][1]
LISA_sc2_HiDensity = igloo_data[1][0]
LISA_sc2_LoDensity = igloo_data[1][1]
LISA_sc3_HiDensity = igloo_data[2][0]
LISA_sc3_LoDensity = igloo_data[2][1]

LISA_sc1_HiDensity_std = igloo_data_std[0][0]
LISA_sc1_LoDensity_std = igloo_data_std[0][1]
LISA_sc2_HiDensity_std = igloo_data_std[1][0]
LISA_sc2_LoDensity_std = igloo_data_std[1][1]
LISA_sc3_HiDensity_std = igloo_data_std[2][0]
LISA_sc3_LoDensity_std = igloo_data_std[2][1]

LISA_sc1_HiDensity_den = density_data[0][0]
LISA_sc1_LoDensity_den = density_data[0][1]
LISA_sc2_HiDensity_den = density_data[1][0]
LISA_sc2_LoDensity_den = density_data[1][1]
LISA_sc3_HiDensity_den = density_data[2][0]
LISA_sc3_LoDensity_den = density_data[2][1]

In [ ]:
# Turn igloo data into long format, so we basically slice 1652 elements at a time and
# attach corresponding velocity in a seperate column, this is why we see the same velocity
# value as the matrix extends downward
igloo_long             = pd.melt(LISA_sc1_HiDensity, id_vars=["ID","I","J","PHI1","PHI2","THETA1","THETA2","PHIavg","THETAavg"], var_name="velocity", value_name="flux")
igloo_long['mass']     = np.tile(np.logspace( -6, 1, 1652), 80)
igloo_long["velocity"] = igloo_long["velocity"].astype("float64")
mommatrix              = vbins.reshape(-1,1)@np.logspace( -6, 1, 1652).reshape(1,-1)
igloo_long

In [ ]:
igloo_long["grunmodel_flux"] = GrunModel(igloo_long["mass"].values) * (365.25*24*3600)
igloo_long

In [ ]:
# Correct the grun value model to include only the value of the flux
# that contains the bins of the masses delineated by 'm'
igloo_long["corrected_grunmodel_flux"] = process_sections(igloo_long["grunmodel_flux"].values, 1652)
igloo_long

In [ ]:
igloo_long["real_flux"] = igloo_long["corrected_grunmodel_flux"]*(igloo_long["flux"]) # introduce an offset of 1?
igloo_long["real_flux"].to_numpy()[0:1000]
#igloo_long

In [ ]:
igloo_long["mass_flux"] = igloo_long["real_flux"] * igloo_long["mass"]
igloo_long

In [ ]:
igloo_long["momentum"] = igloo_long["velocity"] * igloo_long["mass"]
igloo_long["momentum"] = round(igloo_long["momentum"], 2)
igloo_long

In [ ]:
igloo_grouped = igloo_long.groupby(["ID", "I", "J", "PHI1", "PHI2", "THETA1", "THETA2", "PHIavg", "THETAavg", "momentum"])["real_flux"].sum().reset_index()
igloo_grouped

In [ ]:
igloo_wide = igloo_grouped.pivot(index=["ID", "I", "J", "PHI1", "PHI2", "THETA1", "THETA2", "PHIavg", "THETAavg"], columns="momentum", values="real_flux").reset_index()
igloo_wide = igloo_wide.fillna(0)
igloo_wide

In [ ]:
igloo_wide[columnname][0:500]
total_mom

In [ ]:
columns   = igloo_wide.columns[9:]
mombins   = columns.astype("float64").values
total_mom = np.zeros(len(mombins))
for u in range(len(mombins)-1):
    for a, columnname in enumerate(columns):
        for i in range(1652):
                for j in range(79):
                    for k in range(1652):
                        if u == 0:
                            if (mommatrix[j, k] <= mombins[u]):
                                total_mom[u] += igloo_wide[columnname][i]
                        elif u == (len(mombins) - 1):
                            if (mombins[u] <= mommatrix[j, k]):
                                total_mom[u] += igloo_wide[columnname][i]     
                        else: 
                            if (mombins[u] <= mommatrix[j, k] < mombins[u + 1]):
                                total_mom[u] += igloo_wide[columnname][i]     

---

#### Experimental cells

In [ ]:
columns = igloo_wide.columns[9:]

In [ ]:
igloo_long_again

In [ ]:
# Create a data frame that includes the mass generated and the associated flux
# This flux is corrected and scaled 
mass_df = pd.DataFrame({"mass": igloo_long["mass"].to_numpy(), "flux": igloo_long["real_flux"].to_numpy()})

##### Use a "cross" merge to combine the velocity/mass tables, then calc momentum

In [ ]:
mom_flux = pd.merge(igloo_long, mass_df, how="cross", suffixes=("_vel","_mass"))
mom_flux["momentum1"]  = mom_flux["velocity"] * mom_flux["mass"]
mom_flux["mass_flux"]  = mom_flux["real_flux"] * mom_flux["mass"]

In [ ]:
mom_flux

In [ ]:
# Pivot the momentum matrix
wide_mom_flux = mom_flux.pivot_table(index=["ID", "I", "J", "PHI1", "PHI2", "THETA1", "THETA2", "PHIavg", "THETAavg"],
                                     columns="momentum", values="real_flux", fill_value=0)

# Reset the index to make the columns regular columns instead of index levels
wide_mom_flux = wide_mom_flux.reset_index()

In [ ]:
wide_mom_flux

##### TODO: Mass flux needs to be adjusted for the binning we discussed -- Do this before the merge. Also need to normalize flux

---

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 8), dpi = 120, facecolor='w')

vbins              = np.linspace(0.5, 78.5, 79)
size_ofmassarray = 10
bmass = np.logspace(-6, 1, size_ofmassarray)  # grams
G = GrunModel(bmass) * (365.25*24*3600) 

dgrun = np.zeros(size_ofmassarray-1)
for ll in range(size_ofmassarray-1):
    dgrun[ll] = G[ll] - G[ll+1]   # check
    
dgrun /= np.sum(dgrun)
    
numbins = 10
momentumbins = np.logspace(-6, 3, numbins)
momentum_mat = vbins.reshape(-1,1)@bmass.reshape(1,-1)
fluxV     = np.zeros([1652, 79])
fluxM     = np.zeros([1652, 79, size_ofmassarray])
tot       = np.zeros([len(momentumbins)])

for ii in range(1652):
    for jj in range(79): # velocity
        fluxV[:, jj]     = igloo_data[0][0][str(velocity_bins[jj])][ii] # issue when loading the data?
        
for i in range(1652):
    for j in range(79):
        for k in range(size_ofmassarray-1):
            fluxM[:, j, k]     = fluxV[i, j] * dgrun[k]
            
for l in range(numbins - 1):
    for i in range(1652):
        for j in range(79):
            for k in range(size_ofmassarray):
                if l == 0:
                     if (momentum_mat[j, k] <= momentumbins[l]):
                        tot[l]     += fluxM[i, j, k]     
                        
                elif l == (numbins - 1):
                     if (momentumbins[l] <= momentum_mat[j, k]):
                        tot[l]     += fluxM[i, j, k]     
                        
                else: 
                    if (momentumbins[l] <= momentum_mat[j, k] < momentumbins[l + 1]):
                        tot[l]     += fluxM[i, j, k]     

ax.loglog(momentumbins * 1e6, tot, linewidth = 1)
        
plt.xlim([1e-1, 1e8])
plt.ylim([1e-8, 1e6])
plt.ylabel(r'Flux $[numparticles*yr^{-1}m^{-2}]$')
plt.xlabel(r'Momentum, $[\mu N s]$') # newton seconds  
plt.legend()
plt.grid(True)
#plt.savefig('momentum distribution.png',dpi=200)
plt.show()